In [ ]:
import os

import getpass
from pathlib import Path, PureWindowsPath # please check this medium article!! https://medium.com/@ageitgey/python-3-quick-tip-the-easy-way-to-deal-with-file-paths-on-windows-mac-and-linux-11a072b58d5f
import pandas as pd


if getpass.getuser() == 'Konrad':
    project_dir = Path(PureWindowsPath('D:\\DeepLearningProject'))
elif getpass.getuser() == 'fruechtnicht':
    project_dir = Path('/Users/fruechtnicht/NOVA/M.Sc_Data_Science_and_Advanced_Analytics/Semester2/Deep_Learning/Project/project_dir')
elif getpass.getuser() == 'dominika.leszko':
    project_dir = Path(r'C:\Users\dominika.leszko\Desktop\NOVAIMS\SEMESTER2\Deep Learinng\PROJECT\git_repo')
else:
    raise ValueError('Check you own user name and add proper elif statement !!!')
# if you have a windows computer please specify your project path as Konrad, if not as fruechtnicht
print(project_dir)
print(os.chdir(project_dir/ Path('log')))

In [ ]:
dirpath = os.getcwd()
print("current directory is : " + dirpath)

In [ ]:
targets = [
            'ethnic'
            , 
            'age'
            ,
            'gender'
            ]

In [ ]:
import pandas as pd 
import glob
import re
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

sns.set(rc={'figure.figsize':(24,16)})
sns.set_style("white")
sns.set_palette("Reds")

variables = ['nr_of_channel', 'kernel_size', 'stride', 'hidden_neurons']

for target in targets:
    
    for variable in variables:
        final = pd.DataFrame()
        for file in glob.glob("*testing_"+target+"_"+variable+"_*"):
            seed = re.search("\d(?=(_training\.csv))", file).group(0)  
            value = re.search("\d*(?=(_\d*_training\.csv))", file).group(0)  
            df = pd.read_csv(file)
            df[variable]= int(value)
            df['seed']= int(seed)
            final = pd.concat([final,df], axis=0)

        if target == 'age':
            metric = 'val_mean_absolute_error'
        else: 
            metric = 'val_acc'


        n_colors = len(final[variable].unique())

        colors = ['#76b39d','#9c2c2c','#265069','#f59154']
        palette = colors[:n_colors]

        if metric == 'val_mean_absolute_error':

            final[metric] *= -1


        ax = sns.lineplot(x="epoch", y=metric, data=final, hue=variable, linewidth=5.0,legend="full", palette=palette)
        ax.set_title('Performance with different '+variable+ " for "+target, fontsize=40)
        ax.set_xlabel("Epoch",fontsize=40)
        ax.set_ylabel(metric,fontsize=40)
        ax.tick_params(labelsize=45)
        leg = ax.legend(frameon=False, loc='best',ncol=1, fontsize=40,markerscale=5)


        for line in leg.get_lines():
            line.set_linewidth(20.0)
        fig = ax.get_figure()
        plt.savefig(variable+'_'+target+'.png')
        plt.close()



In [ ]:
import re
import pandas as pd 
import glob
import re
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(24,16)})
sns.set_style("white")
sns.set_palette("Reds")

variables = ['2ConvPool', '4ConvPool', 'ConvConvPool']

for target in targets:
    final = pd.DataFrame()
    
    for variable in variables:
        inter = pd.DataFrame()
        for file in glob.glob("*testing_"+target+"_"+variable+"_*"):
            seed = re.search("\d(?=(_training\.csv))", file).group(0)  
            # no parameter value value = re.search("\d*(?=(_\d*_training\.csv))", file) 
            variable = re.search("(?<="+target+"_).*(?=(_\d*_training\.csv))", file).group(0)
            df = pd.read_csv(file)
            df['seed']= int(seed)
            df['Convolution']= str(variable)
            
            inter = pd.concat([inter,df], axis=0)
        final = pd.concat([final,inter], axis=0)
    print(final)

    if target == 'age':
        metric = 'val_mean_absolute_error'
    else: 
        metric = 'val_acc'


    colors = ['#76b39d','#9c2c2c','#265069','#f59154']
    palette = colors[:3]
    
    if metric == 'val_mean_absolute_error':
        
        final[metric] *= -1
        
    ax = sns.lineplot(x="epoch", y=metric, data=final, hue='Convolution', linewidth=5.0,legend="full", palette=palette)
    ax.set_title('Performance with different architectures for '+target, fontsize=40)
    ax.set_xlabel("Epoch",fontsize=40)
    ax.set_ylabel(metric,fontsize=40)
    ax.tick_params(labelsize=45)
    leg = ax.legend(frameon=False, loc='best',ncol=1, fontsize=40,markerscale=5)


    for line in leg.get_lines():
            line.set_linewidth(20.0)
    
    fig = ax.get_figure()
    plt.savefig('Architecture'+'_'+target+'.png')
    plt.close()

# Comparing models## RANK

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
mod1=pd.read_excel('pred_mod1.xlsx')
mod2=pd.read_excel('pred_mod2.xlsx')
mod3=pd.read_excel('pred_mod3.xlsx')
mod4=pd.read_excel('pred_mod4.xlsx')


n_preds=mod1.shape[0]

for mod in [mod1, mod2, mod3, mod4]:
    mod.columns=['gen_true', 'gen_pred', 'ethnic_true', 'ethnic_pred', 'age_true', 'age_pred']

In [ ]:
for mod in [mod1, mod2, mod3, mod4]:
    mod['gen_score']=np.where(mod['gen_true']==mod['gen_pred'], 1, 0)
    mod['ethnic_score']=np.where(mod['ethnic_true']==mod['ethnic_pred'], 1, 0)
    mod['age_score']=np.abs(mod['age_true']-mod['age_pred'])
    

In [ ]:
for mod in [mod1, mod2, mod3, mod4]:
    mod.drop(columns=['gen_true', 'gen_pred', 'ethnic_true', 'ethnic_pred', 'age_true', 'age_pred'], axis=1, inplace=True)

In [ ]:
final_table=pd.DataFrame(columns=['gen_rank', 'ethnic_rank', 'age_rank'])
final_table

In [ ]:
for mod in [mod1, mod2, mod3, mod4]:
    final_table=final_table.append({'gen_rank':np.sum(mod['gen_score']), 'ethnic_rank':np.sum(mod['ethnic_score']),
                        'age_rank':np.average(mod['age_score'])}, ignore_index=True)

final_table_bkp=final_table.copy()

In [ ]:
final_table['gen_rank']=final_table['gen_rank'].rank()
final_table['ethnic_rank']=final_table['ethnic_rank'].rank()
final_table['age_rank']=final_table['age_rank'].rank()
final_table=np.floor(final_table)
final_table.index=['Model_1','Model_2','Model_3','Model_4']
final_table

#ATTENTON! 1 means worst 4 means best! for plotting purposes

#COLORMAP:
#['#FF9473', '#FFE6AC', '#97C977', '#639B94', '#3E757D']

In [ ]:
final_table=pd.read_excel(r'pred_results.xlsx', sheet_name='rank')
final_table.index=final_table['Model']
final_table

In [ ]:
sns.set_style('whitegrid')
final_table.plot(kind='bar', stacked=True, color=['#5A8D87','#E98769','#89B76C'])
plt.title('Models performance on various targets', fontsize=40)
plt.ylabel('Ranking', fontsize=35)
plt.xlabel('')
plt.xticks(fontsize=30, rotation='horizontal')
plt.legend(loc=2, prop={'size': 20})
plt.savefig('rank_compare.png')

# Radar Chart

In [ ]:
final_table_bkp=pd.read_excel(r'pred_results.xlsx', sheet_name='radar')
final_table_bkp.index=final_table['Model']
final_table_bkp.drop(columns=['Model'], axis=1, inplace=True)
final_table_bkp

In [ ]:
data=final_table_bkp.copy()
Attributes=list(data)
AttNo=len(Attributes)
values = data.iloc[2].tolist()#Transfer_learn
values += values [:1]
values2 = data.iloc[1].tolist()#Separate_conv
values2 += values2 [:1]
values3 = data.iloc[0].tolist()#Common_conv
values3 += values3 [:1]
#values4=data.iloc[0].tolist()
#values4+=values4[:1]
angles = [n / float(AttNo) * 2 * pi for n in range(AttNo)]
angles += angles [:1]

In [ ]:
def createRadar(player, data, color):
    Attributes = ["Age","Gender","Race"]
    
    data += data [:1]
    
    angles = [n / 3 * 2 * pi for n in range(3)]
    angles += angles [:1]
    
    ax = plt.subplot(111, polar=True)

    plt.xticks(angles[:-1],Attributes, fontsize=30, fontweight='bold')
    ax.plot(angles,data)
    ax.fill(angles, data, color, alpha=0.3)

    ax.set_title(player, fontsize=40)
    ax.set_ylim(top=0.8)
    plt.show()

In [ ]:
def createRadar4(player, data, player2, data2, player3, data3):
    Attributes = ["Age","Gender","Race"]
    font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 100}

    
    
    data += data[:1]
    data2 += data2[:1]
    data3 += data3[:1]
    
    angles = [n / 3 * 2 * pi for n in range(3)]
    
    angles += angles [:1]
    
    angles2 = [n / 3 * 2 * pi for n in range(3)]
    angles2 += angles2 [:1]
    
    angles3 = [n / 3 * 2 * pi for n in range(3)]
    angles3 += angles3 [:1]

    
    ax = plt.subplot(111, polar=True)

    #Create the chart as before, but with both Ronaldo's and Messi's angles/values
    ax = plt.subplot(111, polar=True)
    
    plt.xticks(angles[:-1],Attributes, fontsize=30, fontweight='bold')

    ax.plot(angles,values, color='#9c2c2c')
    ax.fill(angles, values, '#9c2c2c', alpha=0.3)

    ax.plot(angles2,values2, color='#76b39d')
    ax.fill(angles2, values2, '#76b39d', alpha=0.3)
    
    ax.plot(angles3,values3, color='#265069')
    ax.fill(angles3, values3, '#265069', alpha=0.3)
    plt.rc('font', **font)
    
    
    #Rather than use a title, individual text points are added
    
    plt.figtext(0.2,0.9,player,color="#9c2c2c", fontweight='bold', fontsize=25)
    plt.figtext(0.2,0.8,player2,color="#76b39d", fontweight='bold', fontsize=25)
    plt.figtext(0.2,0.7,player3,color="#265069", fontweight='bold', fontsize=25)
    plt.savefig('Radar_plot.png')
    plt.show()

In [ ]:
createRadar4("Common_conv", [0.42, 0.68, 0.45], 'Separate_conv', [0.44, 0.8, 0.55],'Transfer_learn', [0.13, 0.53, 0.41])

In [ ]:
createRadar("Common_conv", [0.42, 0.68, 0.45], color='#265069')

In [ ]:
createRadar("Separate_conv", [0.44, 0.8, 0.55], color='#76b39d')

In [ ]:
createRadar("Transfer_learn", [0.13, 0.53, 0.41], color='#9c2c2c')